In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
smtp_server = SMTPTestServer(email_server)
smtp_server.start()

In [ ]:
# stdlib
import random

# syft absolute
import syft as sy
from syft.service.job.job_stash import Job

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
# third party
from helpers import load_users
from job_helpers import load_jobs
from job_helpers import save_jobs

In [ ]:
users = load_users(high_client)

In [ ]:
jobs = load_jobs(users, high_client)

In [ ]:
all_requests = high_client.requests

In [ ]:
def get_request_for_job(requests, job):
    job_requests = [
        r for r in all_requests if r.code.service_func_name == job.func_name
    ]
    if len(job_requests) == 1:
        return job_requests[0]
    if len(job_requests) > 1:
        raise Exception(f"Multiple of the same job: {job} in requests: {requests}")
    return None

In [ ]:
submitted_jobs = [job for job in jobs if job.is_submitted]

In [ ]:
def approve_by_running(request):
    job = request.code(blocking=False)
    result = job.wait()
    assert result
    job_info = job.info(result=True)
    # need force when running multiple times
    # todo check and dont run if its already done
    response = request.deposit_result(job_info, approve=True, force=True)
    return response

In [ ]:
# TODO we should record whether it was approved or deposited
# and test doing both in either order as there might be a bug when
# force overwriting
# also changing deny to approve and back again

In [ ]:
for job in submitted_jobs:
    request = get_request_for_job(all_requests, job)
    choice = None
    number_of_emails = len(
        email_server.get_emails_for_user(request.requesting_user_email)
    )
    if job.should_succeed:
        if random.randrange(2):
            choice = "approved with deposit_result"
            response = approve_by_running(request)
            assert isinstance(response, Job)
        else:
            choice = "approved"
            response = request.approve()
            assert isinstance(response, sy.SyftSuccess)
    else:
        choice = "denied"
        response = request.deny(
            reason=f"Your request {job.func_name} looks wrong, try again."
        )
        assert isinstance(response, sy.SyftSuccess)

    after_users_emails = len(
        email_server.get_emails_for_user(request.requesting_user_email)
    )
    # assert after_users_emails > number_of_emails
    job.admin_reviewed = True
    print(f"Job {job.func_name} should succeed: {job.should_succeed} and was {choice}")

In [ ]:
save_jobs(jobs)

In [ ]:
smtp_server.stop()

In [ ]:
server.land()